In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
#set path for my_tools script
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap
from my_tools import file_dic, plot_dic
import matplotlib.ticker as plticker
from mpl_toolkits.basemap import Basemap, addcyclic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
path = '/media/onno/Algemeen/Thesis/'

In [5]:
#list filenames
file_pers_hw = 'GFS_T850/persistent_hw_lon_6_12_lat_52_46.npy'
file_pers_cw = 'GFS_T850/persistent_cw_lon_6_12_lat_52_46.npy'
file_short_hw = 'GFS_T850/short_hw_lon_6_12_lat_52_46.npy'
file_short_cw = 'GFS_T850/short_cw_lon_6_12_lat_52_46.npy'
file_ind_short_hw = 'GFS_T850/independent_short_hw_lon_6_12_lat_52_46.npy'
file_ind_short_cw = 'GFS_T850/independent_short_cw_lon_6_12_lat_52_46.npy'

#open numpy array with extreme events
pers_hw = np.load(path+file_pers_hw)
pers_cw = np.load(path+file_pers_cw)
ind_short_hw = np.load(path+file_ind_short_hw)
ind_short_cw = np.load(path+file_ind_short_cw)
#load independent short extremes only
short_hw = np.load(path+file_short_hw)[ind_short_hw,:]
short_cw = np.load(path+file_short_cw)[ind_short_cw,:]



In [16]:
file_env = 'era51_mars_env_wledit2000-10000_latavg_v300_79-19_6hourly_smoothed.nc'
file_cp = 'era51_mars_phasevel_wledit2000-10000_latavg_v300_envgt15_79-19_6hourly_setvrange_-100to100.nc'
file_t850 = 'era51_mars_t850_79-19_6hourly_anom_from_smoothed04_clim.nc'
file_v300 = 'era51_mars_v300_wledit2000-10000_latavg_79-19_6hourly.nc'
file_z500 = 'era51_mars_phi500_79-19_6hourly.nc'
file_mslp = 'era5_mslp_79-19_6hourly_remapbil2x2.nc'

era_env = xr.open_dataset(path+file_env)
era_cp = xr.open_dataset(path+file_cp)
era_v300 = xr.open_dataset(path+file_v300)
era_z500 = xr.open_dataset(path+file_z500)
era_mslp = xr.open_dataset(path+file_mslp)
era_t850 = xr.open_dataset(path+file_t850)

Composite Event onset only

In [11]:
#define color map
cmap = ncl_colormap()
#Define axis labels
extreme_labelz = ['Persistent Warm Extremes','Persistent Cold Extremes',
                 'Short-Lived Warm Extremes','Short-Lived Cold Extremes']
#Put all extreme dates in list to loop over them
extremez = [pers_hw,pers_cw,short_hw,short_cw]
#Create 2x2 figure 
fig,axz = plt.subplots(2,2,figsize=(8,7))
for i in range(4):
    #select figure axis
    ax = axz.flat[i]
    #Pick relevant dates and only event onsets
    datez = extremez[i][:,0]
    #make composite analysis by averaging over all event onsets
    ds_env = era_env.sel(time=datez,latitude=slice(85,0)).mean(dim='time').squeeze()
    ds_cp = era_cp.sel(time=datez,latitude=slice(85,0)).mean(dim='time').squeeze()
    #create cyclic dataset
    ds_env_cyclic, lon_env_cyclic = addcyclic(ds_env.v.values, ds_env.longitude)
    ds_cp_cyclic, lon_cp_cyclic = addcyclic(ds_cp.v.values, ds_cp.longitude)
    lon,lat = np.meshgrid(lon_env_cyclic,ds_env.latitude)
    #draw map with lambert projection
    m = Basemap(projection='lcc',lat_0=50,lon_0=10,resolution='c',ax=ax,
                llcrnrlon=-40,llcrnrlat=15,urcrnrlon=90,urcrnrlat=75)
    #draw contour with contour lines
    im = m.contourf(lon,lat,ds_cp_cyclic,latlon='true', extend='both', cmap=cmap,
                   vmin=0,vmax=10,levels=np.linspace(0,10,101))
    cs = m.contour(lon,lat,ds_env_cyclic,latlon='true',colors='dimgrey',levels=np.linspace(0,60,21))
    #draw contour line labels
    ax.clabel(cs,inline=True,fmt='%2.0f',colors='k')
    #set axis titles
    ax.set_title(extreme_labelz[i])
    #draw continents
    m.drawcoastlines()

#create extra axis for color bar and draw color bar
cbax = fig.add_axes([0.2,0.07,0.6,0.015])

cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(0,10,11))
cbar.set_label('Cp (m/s)')
fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0.05,hspace=0.15)     
fig.savefig(path+'Main_Figures/composite_env_cp_onset')
plt.close(fig)
    
    

/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[t

Composite complete event

In [14]:
#define color map
cmap = ncl_colormap()
#Define axis labels
extreme_labelz = ['Persistent Warm Extremes','Persistent Cold Extremes',
                 'Short-Lived Warm Extremes','Short-Lived Cold Extremes']
#Put all extreme dates in list to loop over them
extremez = [pers_hw,pers_cw,short_hw,short_cw]
#Create 2x2 figure 
fig,axz = plt.subplots(2,2,figsize=(8,7))
for i in range(4):
    #select figure axis
    ax = axz.flat[i]
    #Pick relevant dates and only event onsets
    datez = extremez[i]
    #make composite analysis by averaging over all event onsets
    ds_env_list = []
    ds_cp_list = []
    for date in datez:
        ds_env_sub = era_env.sel(time=slice(date[0],date[1]),latitude=slice(85,0)).squeeze()
        ds_cp_sub = era_cp.sel(time=slice(date[0],date[1]),latitude=slice(85,0)).squeeze()
        ds_env_list.append(ds_env_sub)
        ds_cp_list.append(ds_cp_sub)
    ds_env = xr.concat(ds_env_list,dim='time').mean(dim='time')
    ds_cp = xr.concat(ds_cp_list,dim='time').mean(dim='time')
    #create cyclic dataset
    ds_env_cyclic, lon_env_cyclic = addcyclic(ds_env.v.values, ds_env.longitude)
    ds_cp_cyclic, lon_cp_cyclic = addcyclic(ds_cp.v.values, ds_cp.longitude)
    lon,lat = np.meshgrid(lon_env_cyclic,ds_env.latitude)
    #Draw map with lambert projection
    m = Basemap(projection='lcc',lat_0=50,lon_0=10,resolution='c',ax=ax,
                llcrnrlon=-40,llcrnrlat=15,urcrnrlon=90,urcrnrlat=75)
    #draw contour with contour lines
    im = m.contourf(lon,lat,ds_cp_cyclic,latlon='true', extend='both', cmap=cmap,
                   vmin=0,vmax=10,levels=np.linspace(0,10,101))
    cs = m.contour(lon,lat,ds_env_cyclic,latlon='true',colors='dimgrey',levels=np.linspace(0,60,21))
    #draw contour line labels
    ax.clabel(cs,inline=True,fmt='%2.0f',colors='k')
    #set axis titles
    ax.set_title(extreme_labelz[i])
    #draw continents
    m.drawcoastlines()

#create extra axis for color bar and draw color bar
cbax = fig.add_axes([0.2,0.07,0.6,0.015])

cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(0,10,11))
cbar.set_label('Cp (m/s)')
fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0.05,hspace=0.15)     
fig.savefig(path+'Main_Figures/composite_env_cp_all')
plt.close(fig)


/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[t

Composite event middle event day only 

In [15]:
#define color map
cmap = ncl_colormap()
#Define axis labels
extreme_labelz = ['Persistent Warm Extremes','Persistent Cold Extremes',
                 'Short-Lived Warm Extremes','Short-Lived Cold Extremes']
#Put all extreme dates in list to loop over them
extremez = [pers_hw,pers_cw,short_hw,short_cw]
#Create 2x2 figure 
fig,axz = plt.subplots(2,2,figsize=(8,7))
for i in range(4):
    #select figure axis
    ax = axz.flat[i]
    #Pick relevant dates and only event onsets
    datez = extremez[i]
    datez_middle = [datez[i,0]+((datez[i,1]+pd.Timedelta(1,'days'))-datez[i,0])/2 for i in range(len(datez))]
    #make composite analysis by averaging over all event onsets
    ds_env = era_env.sel(time=datez_middle,latitude=slice(85,0)).mean(dim='time').squeeze()
    ds_cp = era_cp.sel(time=datez_middle,latitude=slice(85,0)).mean(dim='time').squeeze()
    #create cyclic dataset
    ds_env_cyclic, lon_env_cyclic = addcyclic(ds_env.v.values, ds_env.longitude)
    ds_cp_cyclic, lon_cp_cyclic = addcyclic(ds_cp.v.values, ds_cp.longitude)
    lon,lat = np.meshgrid(lon_env_cyclic,ds_env.latitude)
    m = Basemap(projection='lcc',lat_0=50,lon_0=10,resolution='c',ax=ax,
                llcrnrlon=-40,llcrnrlat=15,urcrnrlon=90,urcrnrlat=75)
    #Draw contour plot with contour lines
    im = m.contourf(lon,lat,ds_cp_cyclic,latlon='true', extend='both', cmap=cmap,
                   vmin=0,vmax=10,levels=np.linspace(0,10,101))
    cs = m.contour(lon,lat,ds_env_cyclic,latlon='true',colors='dimgrey',levels=np.linspace(0,60,21))
    #draw contour line labels
    ax.clabel(cs,inline=True,fmt='%2.0f',colors='k')
    #set axis titles
    ax.set_title(extreme_labelz[i])
    #draw continents
    m.drawcoastlines()

#create extra axis for color bar and draw color bar
cbax = fig.add_axes([0.2,0.07,0.6,0.015])

cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(0,10,11))
cbar.set_label('Cp (m/s)')
fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0.05,hspace=0.15)         
fig.savefig(path+'Main_Figures/composite_env_cp_middle')
plt.close(fig)
      
    

/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[t

Plot temperature anomalie with 300 hpa meridional wind

In [21]:
#define color map
cmap = ncl_colormap()
#Define axis labels
extreme_labelz = ['Persistent Warm Extremes','Persistent Cold Extremes',
                 'Short-Lived Warm Extremes','Short-Lived Cold Extremes']
#Put all extreme dates in list to loop over them
extremez = [pers_hw,pers_cw,short_hw,short_cw]
#Create 2x2 figure 
fig,axz = plt.subplots(2,2,figsize=(8,7))
for i in range(4):
    #select figure axis
    ax = axz.flat[i]
    #Pick relevant dates and only event onsets
    datez = extremez[i][:,0]
    #make composite analysis by averaging over all event onsets
    ds_t850 = era_t850.sel(time=datez,latitude=slice(85,0)).mean(dim='time').squeeze()
    ds_v300 = era_v300.sel(time=datez,latitude=slice(85,0)).mean(dim='time').squeeze()
    #create cyclic dataset
    ds_t850_cyclic, lon_t850_cyclic = addcyclic(ds_t850.t.values, ds_t850.longitude)
    ds_v300_cyclic, lon_v300_cyclic = addcyclic(ds_v300.v.values, ds_v300.longitude)
    lon,lat = np.meshgrid(lon_t850_cyclic,ds_t850.latitude)
    #create map with lambert projection
    m = Basemap(projection='lcc',lat_0=50,lon_0=10,resolution='c',ax=ax,
                llcrnrlon=-40,llcrnrlat=15,urcrnrlon=90,urcrnrlat=75)
    im = m.contourf(lon,lat,ds_t850_cyclic,latlon='true', extend='both', cmap='bwr',
                   vmin=-10,vmax=10,levels=np.linspace(-10,10,201))
    #create contour labels
    cs = m.contour(lon,lat,ds_v300_cyclic,latlon='true',colors='dimgrey',levels=np.linspace(-40,40,21))
    ax.clabel(cs,inline=True,fmt='%2.0f',colors='k')
    ax.set_title(extreme_labelz[i])
    #draw continents
    m.drawcoastlines()

#make room for colorbar and create one
cbax = fig.add_axes([0.2,0.07,0.6,0.015])

cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(-10,10,5))
cbar.set_label('850 hPa temperature Anomaly (K)')
fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0.05,hspace=0.15)     
fig.savefig(path+'Main_Figures/composite_t850_v300_onset')
plt.close(fig)
    
    

/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[t

Plot temperature anomalie with 300 hpa meridional wind

In [22]:
#define color map
cmap = ncl_colormap()
#Define axis labels
extreme_labelz = ['Persistent Warm Extremes','Persistent Cold Extremes',
                 'Short-Lived Warm Extremes','Short-Lived Cold Extremes']
#Put all extreme dates in list to loop over them
extremez = [pers_hw,pers_cw,short_hw,short_cw]
#Create 2x2 figure 
fig,axz = plt.subplots(2,2,figsize=(8,7))
for i in range(4):
    #select figure axis
    ax = axz.flat[i]
    #Pick relevant dates and only event onsets
    datez = extremez[i]
    datez_middle = [datez[i,0]+((datez[i,1]+pd.Timedelta(1,'days'))-datez[i,0])/2 for i in range(len(datez))]
    #make composite analysis by averaging over all event onsets
    ds_t850 = era_t850.sel(time=datez_middle,latitude=slice(85,0)).mean(dim='time').squeeze()
    ds_v300 = era_v300.sel(time=datez_middle,latitude=slice(85,0)).mean(dim='time').squeeze()
    #create cyclic dataset
    ds_t850_cyclic, lon_t850_cyclic = addcyclic(ds_t850.t.values, ds_t850.longitude)
    ds_v300_cyclic, lon_v300_cyclic = addcyclic(ds_v300.v.values, ds_v300.longitude)
    lon,lat = np.meshgrid(lon_t850_cyclic,ds_t850.latitude)
    #create map with lambert projection
    m = Basemap(projection='lcc',lat_0=50,lon_0=10,resolution='c',ax=ax,
                llcrnrlon=-40,llcrnrlat=15,urcrnrlon=90,urcrnrlat=75)
    im = m.contourf(lon,lat,ds_t850_cyclic,latlon='true', extend='both', cmap='bwr',
                   vmin=-10,vmax=10,levels=np.linspace(-10,10,201))
    #create contour labels
    cs = m.contour(lon,lat,ds_v300_cyclic,latlon='true',colors='dimgrey',levels=np.linspace(-40,40,21))
    ax.clabel(cs,inline=True,fmt='%2.0f',colors='k')
    ax.set_title(extreme_labelz[i])
    #draw continents
    m.drawcoastlines()

#make room for colorbar and create one
cbax = fig.add_axes([0.2,0.07,0.6,0.015])

cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(-10,10,5))
cbar.set_label('850 hPa temperature Anomaly (K)')
fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0.05,hspace=0.15)     
fig.savefig(path+'Main_Figures/composite_t850_v300_middle')
plt.close(fig)
    
    

/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[t

In [32]:
#define color map
cmap = ncl_colormap()
#Define axis labels
extreme_labelz = ['Persistent Warm Extremes','Persistent Cold Extremes',
                 'Short-Lived Warm Extremes','Short-Lived Cold Extremes']
#Put all extreme dates in list to loop over them
extremez = [pers_hw,pers_cw,short_hw,short_cw]
#Create 2x2 figure 
fig,axz = plt.subplots(2,2,figsize=(8,7))
for i in range(4):
    #select figure axis
    ax = axz.flat[i]
    #Pick relevant dates and only event onsets
    datez = extremez[i]
    datez_middle = [datez[i,0]+((datez[i,1]+pd.Timedelta(1,'days'))-datez[i,0])/2 for i in range(len(datez))]
    #make composite analysis by averaging over all event onsets
    ds_t850 = era_t850.sel(time=datez_middle,latitude=slice(85,0)).mean(dim='time').squeeze()
    ds_z500 = era_z500.sel(time=datez_middle,latitude=slice(85,0)).mean(dim='time').squeeze()
    #create cyclic dataset
    ds_t850_cyclic, lon_t850_cyclic = addcyclic(ds_t850.t.values, ds_t850.longitude)
    #Divide 500 hPa geopential height by 100 to gain dam
    ds_z500_cyclic, lon_z500_cyclic = addcyclic(ds_z500.z.values/100, ds_z500.longitude)
    lon,lat = np.meshgrid(lon_t850_cyclic,ds_t850.latitude)
    #create map with lambert projection
    m = Basemap(projection='lcc',lat_0=50,lon_0=10,resolution='c',ax=ax,
                llcrnrlon=-40,llcrnrlat=15,urcrnrlon=90,urcrnrlat=75)
    im = m.contourf(lon,lat,ds_t850_cyclic,latlon='true', extend='both', cmap='bwr',
                   vmin=-10,vmax=10,levels=np.linspace(-10,10,201))
    #create contour labels
    cs = m.contour(lon,lat,ds_z500_cyclic,latlon='true',colors='dimgrey',levels=np.linspace(400,600,41))
    ax.clabel(cs,inline=True,fmt='%2.0f',colors='k')
    ax.set_title(extreme_labelz[i])
    #draw continents
    m.drawcoastlines()

#make room for colorbar and create one
cbax = fig.add_axes([0.2,0.07,0.6,0.015])

cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(-10,10,5))
cbar.set_label('850 hPa temperature Anomaly (K)')
fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0.05,hspace=0.15)     
fig.savefig(path+'Main_Figures/composite_t850_z500_middle')
plt.close(fig)
    
    

/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/.local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:5133: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[t